In [15]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as ms
import sys

from IPython.display import display, Markdown
from ipywidgets import Layout

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery
 
client = BigQuery.getBigQueryClient_TDMScenarios()

In [16]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [17]:
# FILTER
#strSQLWhere = ' WHERE NOT (t.scenarioID = 33)'
strSQLWhere = ''

# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share AS t" + strSQLWhere).to_dataframe()
#display(dfTransitSummary)

dfRouteSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS t" + strSQLWhere).to_dataframe()
#display(dfRouteSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios AS t" + strSQLWhere).to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge two dataframes
dfRouteSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfRouteSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData


In [18]:
dfScenarios

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
0,8.01,v9,WF TDM v9.0 - 2023-04-14,BY_2019,BY,2019
1,8.02,v9,WF TDM v9.0 - 2023-04-14,BY_2023,BY,2023
2,8.13,v9,WF TDM v9.0 - 2023-04-14,RTP_2032,RTP,2032
3,8.15,v9,WF TDM v9.0 - 2023-04-14,RTP_2042,RTP,2042
4,8.16,v9,WF TDM v9.0 - 2023-04-14,RTP_2050,RTP,2050
5,8.18,v9,WF TDM v9.0 - 2023-04-14,TIP_2028,TIP,2028
6,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019
7,7.01,v9,WF TDM v9.0 - 2023-04-14,BY_2019,BY,2019
8,3.02,v9,WF TDM v9.0 - 2022-12-19,BY_2023,BY,2023
9,7.02,v9,WF TDM v9.0 - 2023-04-14,BY_2023,BY,2023


In [14]:
# check to see if all scenarios has same record count
dfTransitSummaryPlotData_Check = dfTransitSummaryPlotData
dfTransitSummaryPlotData_Check.groupby(['tdmVersionWithDate','scenarioGroup','scenarioYear']).agg(COUNT=('TRIPS','count'))

COUNT
tdmVersionWithDate          scenarioGroup scenarioYear       
WF TDM v8.3.2 - 2022-02-04a BY            2019            280
                            RTP           2030            280
                                          2040            280
                                          2050            280
                            TIP           2024            280
WF TDM v8.3.2 - 2022-09-21  BY            2019            280
                                          2023            280
                            Needs         2032            280
                                          2042            280
                                          2050            280
                            NoBuild       2032            280
                                          2042            280
                                          2050            280
                            RTP           2032            280
                                          2042            280
                                          2050            280
                            TIP           2028            280
WF TDM v8.3.2 - 2023-02-28  BY            2019            280
                                          2023            280
                            RTP           2032            280
                                          2042            280
                                          2050            280
                            TIP           2028            280
WF TDM v9.0 - 2022-12-19    BY            2019            280
                                          2023            280
                            EF_RTP        2032            280
                                          2042            280
                                          2050            280
                            Needs         2032            280
                                          2042            280
                                          2050            280
                            NoBuild       2032            280
                                          2042            280
                                          2050            280
                            RTP           2032            280
                                          2042            280
                                          2050            280
                            TIP           2028            280
WF TDM v9.0 - 2023-04-14    BY            2019            560
                                          2023            560
                            EF_RTP        2032            280
                                          2042            280
                                          2050            280
                            NoBuild       2032            280
                                          2042            280
                                          2050            280
                            RTP           2032            560
                                          2042            560
                                          2050            560
                            TIP           2028            560

In [12]:
#PLOTTING FUNCTION

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):
    
    #output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
            
        data = []

        for v in tdmVersionsWithDate:
            for g in scenarioGroups:
                for m in modeGroups:

                    # only do if data in dataframe since BY data is concatonated later
                    if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                        # data for plotting from filtered dataframe}
                        plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v                         ) &
                                                            (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [g])) &
                                                            (dfTransitSummaryPlotData['modeGroup'         ]==m                         ) &
                                                            (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)            ) &
                                                            (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)              )]

                        #display(plotdata)

                        plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                        # fill any NaN values with zeros
                        plotdata = plotdata.fillna(0)

                        #display(plotdata)

                        xplot = plotdata['scenarioYear']
                        yplot = plotdata['TRIPS'       ]

                        trace1 = go.Scatter(
                            x=xplot,
                            y=yplot,
                            mode='markers+lines',
                            name= v + ' ' + g + ' ' + m,
                            marker=dict(size=12,
                                    line=dict(width=2,
                                                color='DarkSlateGrey'))#,
                            #stackgroup='one',
                            #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            #                line=dict(
            #                    shape='spline'
            #                )
                        )
                        data.append(trace1)


        layout = go.Layout(
            title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
            yaxis=dict(
                title='Trips',
                rangemode = 'tozero'#,
                #range=(0,np.null)
            ),
            xaxis=dict(
                title='Year',
                range=(2018,2051)
            ),
            width=1000,
            height=450
        )
    
        fig = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig)

In [13]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTdmVersionWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups     = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes              = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp           = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod             = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionsWithDate = widgets.SelectMultiple(options=lstTdmVersionWithDate, value=(lstTdmVersionWithDate), description='TDM Version'   )
selectScenarioGroups      = widgets.SelectMultiple(options=lstScenarioGroups    , value=('RTP',               ), description='Scenario Group')
selectModes               = widgets.SelectMultiple(options=lstModes             , value=('4: Transit',        ), description='Mode Group'    )
selectTripPurps           = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp          ), description='Purpose'       )
selectPeriods             = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod            ), description='Periods'       )

# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionsWithDate, selectScenarioGroups, selectModes, selectTripPurps, selectPeriods])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot, {'tdmVersionsWithDate':selectTdmVersionsWithDate, 'scenarioGroups':selectScenarioGroups, 'modeGroups':selectModes, 'trippurps':selectTripPurps, 'periods':selectPeriods})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)

#ADD RAW OBSERVED CURVES

# Trip Mode Split Interactive Chart

Output()

Output()

In [8]:
hbox

In [ ]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods, percentOrTotal):
    
    output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
        
        data = []

        modeGroups = sorted(modeGroups)

        for m in modeGroups: 
            # only do if data in dataframe since BY data is concatonated later
            if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) &
                                        (dfTransitSummaryPlotData['scenarioGroup'     ]==scenarioGroup     ) &
                                        (dfTransitSummaryPlotData['modeGroup'         ]==m                 ) &
                                        (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)    ) &
                                        (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)      )].shape[0]>1:

                # data for plotting from filtered dataframe}
                plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                    (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                    (dfTransitSummaryPlotData['modeGroup'         ]==m                                     ) &
                                                    (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)                        ) &
                                                    (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)                          )]

                #display(plotdata)

                plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                #display(plotdata)

                xplot = plotdata['scenarioYear']
                yplot = plotdata['TRIPS'       ]

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name= m,
                    stackgroup='one'#,
                    #groupnorm='percent' # sets the normalization for the sum of the stackgroup
                )
                if percentOrTotal=='Share':
                    trace1.groupnorm = 'percent'
                data.append(trace1)


        layout = go.Layout(
            title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
            yaxis=dict(
                title='Trips'#,
                #rangemode = 'tozero',
                #ticksuffix='%',
                #range=(0,100)
            ),
            xaxis=dict(
                title='Year'#,
                #range=(2018,2051)
            ),
            width=1000,
            height=600
        )
        
        fig2 = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig2)
        #display(fig2)

        #widgets_changed = False



In [ ]:
# MAKE INTERACTIVE CHART
#py.offline.init_notebook_mode(connected=True)

lstTdmVersionsWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes               = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp            = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod              = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                    ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectModes              = widgets.SelectMultiple(options=lstModes             , value=(lstModes[7:13]           ), description='Mode Group'    , layout=Layout(display="flex", flex_flow='row'))
selectTripPurps          = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp              ), description='Purpose'       , layout=Layout(display="flex", flex_flow='row'))
selectPeriods            = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod                ), description='Periods'       , layout=Layout(display="flex", flex_flow='row'))
selectPercentOrTotal     = widgets.Select        (options=['Share','Total']    , value=('Share'                  ), description='Stack Type'    , layout=Layout(display="flex", flex_flow='row'))

# Set the width of the widget
selectTdmVersionWithDate.layout.width = '350px'
selectScenarioGroup     .layout.width = '150px'
selectModes             .layout.width = '250px'
selectTripPurps         .layout.width = '150px'
selectPeriods           .layout.width = '120px'
selectPercentOrTotal    .layout.width = '180px'

# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup, selectModes, selectTripPurps, selectPeriods, selectPercentOrTotal])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot_stackedarea, {'tdmVersionWithDate': selectTdmVersionWithDate, 'scenarioGroup':selectScenarioGroup, 'modeGroups':selectModes, 'trippurps':selectTripPurps, 'periods':selectPeriods, 'percentOrTotal':selectPercentOrTotal})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)
#ADD RAW OBSERVED CURVES

# Trip Mode Split Interactive Chart

Output()

Output()

# Boardings

In [ ]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea_boardings(tdmVersionWithDate, scenarioGroup, percentOrTotal):
    
    output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
            
        data = []

        modes = sorted(dfRouteSummaryWithScenarioData['MODE'].unique())

        for m in modes: 
            # only do if data in dataframe since BY data is concatonated later
            if dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate) &
                                            (dfRouteSummaryWithScenarioData['scenarioGroup'     ]==scenarioGroup     ) &
                                            (dfRouteSummaryWithScenarioData['MODE'              ]==m                 )].shape[0]>1:

                # data for plotting from filtered dataframe}
                plotdata = dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                        (dfRouteSummaryWithScenarioData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                        (dfRouteSummaryWithScenarioData['MODE'              ]==m                                     )]

                #display(plotdata)

                plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(BOARDINGS=('DY_BRDA','sum'))

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                #display(plotdata)

                xplot = plotdata['scenarioYear']
                yplot = plotdata['BOARDINGS'   ]

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name= 'Mode ' + str(m),
                    stackgroup='one'#,
                    #groupnorm='percent' # sets the normalization for the sum of the stackgroup
                )
                if percentOrTotal=='Share':
                    trace1.groupnorm = 'percent'
                data.append(trace1)


        layout = go.Layout(
            title=tdmVersionWithDate + ' Boardings by Mode',
            yaxis=dict(
                title='Trips'#,
                #rangemode = 'tozero',
                #ticksuffix='%',
                #range=(0,100)
            ),
            xaxis=dict(
                title='Year'#,
                #range=(2018,2051)
            ),
            width=1000,
            height=400
        )
        
        fig2 = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig2)


In [ ]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

from ipywidgets import Layout

lstTdmVersionsWithDate = sorted(dfRouteSummaryWithScenarioData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfRouteSummaryWithScenarioData['scenarioGroup'     ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                    ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectPercentOrTotal     = widgets.Select        (options=['Share','Total']    , value=('Share'                  ), description='Stack Type'    , layout=Layout(display="flex", flex_flow='row'))

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup])


# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup, selectPercentOrTotal])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot_stackedarea_boardings, {'tdmVersionWithDate': selectTdmVersionWithDate, 'scenarioGroup':selectScenarioGroup, 'percentOrTotal':selectPercentOrTotal})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)


# Trip Mode Split Interactive Chart

Output()

Output()

In [ ]:
dfRouteSummaryWithScenarioData

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear,MODE,NAMEID,NAME,OPERATOR,...,DY_D8_BRDA,DY_W8_BRDA,DY_8_XITB,DY_D9_BRDA,DY_W9_BRDA,DY_9_XITB,DY_D_BRDA,DY_W_BRDA,DY_BRDA,DY_XITB
0,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,5,88.0,S1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.71,43.16,43.87,43.87
1,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,47.0,S4,0,...,0.00,0.00,0.00,0.00,0.00,0.00,36.12,1307.91,1344.03,1344.03
2,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,5,88.1,S1-,0,...,0.00,0.00,0.00,0.03,2.52,2.55,1.95,1437.13,1439.08,1439.08
3,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,38.0,S17,0,...,0.00,0.06,0.06,0.00,3.73,3.73,2.02,490.75,492.77,492.77
4,3.05,v9,WF TDM v9.0 - 2022-12-19,EF_RTP_2050,EF_RTP,2050,4,43.0,S31,0,...,0.00,0.00,0.00,0.00,0.00,0.00,7.54,163.55,171.09,171.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9820,7.18,v9,WF TDM v9.0 - 2023-04-14,TIP_2028,TIP,2028,4,7.1,M823_5RY2023-,0,...,6.32,8.52,14.84,0.05,3.31,3.36,9.79,110.61,120.40,120.40
9821,7.18,v9,WF TDM v9.0 - 2023-04-14,TIP_2028,TIP,2028,4,1.0,M805_Santaquin,0,...,1.77,11.92,13.69,0.04,7.00,7.04,166.18,94.02,260.20,260.20
9822,7.18,v9,WF TDM v9.0 - 2023-04-14,TIP_2028,TIP,2028,4,12.0,M850_StateStreet,0,...,25.53,176.07,201.60,3.37,343.22,346.59,265.06,3021.74,3286.80,3286.80
9823,7.18,v9,WF TDM v9.0 - 2023-04-14,TIP_2028,TIP,2028,4,15.0,M871_SLtoUtahCnty,0,...,18.00,57.97,75.97,0.00,0.00,0.00,21.57,113.74,135.31,135.30
